In [1]:
%%capture
cd ..

In [2]:
%%capture
!pip install uci-dataset

In [ ]:
import matplotlib.pyplot as plt
import uci_dataset as dataset
from sklearn import model_selection, metrics
from sklearn.preprocessing import OrdinalEncoder
from palma import Project, ModelEvaluation, ModelSelector
from palma.components import ScoringAnalysis, ShapAnalysis
from palma.utils import plotting

In [ ]:
data = dataset.load_credit_approval()
X = data.drop(columns="A16")
cat = ["A13", "A4", "A6", "A7", "A5"]
y = data["A16"] == "+"
X["A12"] = X["A12"] == "t"
X["A9"] = X["A9"] == "t"
X["A10"] = X["A10"] == "t"
X["A1"] = X["A1"] == "a"
X[cat] = OrdinalEncoder().fit_transform(X[cat])

**Create the project**
To create vanilla project (without components), only input data $X$, target data $y$ and validation method needs to be provided

In [ ]:
project = Project(problem="classification", project_name="test")
project.start(
    X, y,
    splitter=model_selection.ShuffleSplit(n_splits=4, random_state=42))

Here are representation of the validation strategy
* For validation steps
* For test steps

In [ ]:
res = plotting.plot_splitting_strategy(
    project.X, project.y, 
    iter_cross_validation=project.validation_strategy.indexes_val,
    cmap="rainbow_r")

In [ ]:
res = plotting.plot_splitting_strategy(
    project.X, project.y, 
    iter_cross_validation=project.validation_strategy.indexes_train_test,
    cmap="rainbow_r")

**Run Opimization** 
Using the FLAML library, the best model (in a given estimator family) is searched

In [ ]:
%%capture
ms = ModelSelector(engine="FlamlOptimizer",
                    engine_parameters=dict(time_budget=10))
ms.start(project)

In [ ]:
ms.best_model_

**Analyse model** Once the best parameters and model is found, use the `ModelEvaluation` api to assess performance. Several built-in component are provided :
- `ScoringAnalysis`
- `RegressionAnalysis`
- `ShapAnalysis`

They can be related to either validation indexes (optimized by `ModelSelector`) or train/test indexes 

In [ ]:
%%capture
model = ModelEvaluation(estimator=ms.best_model_)
model.add(ScoringAnalysis(on="indexes_val"))
model.add(ShapAnalysis(on="indexes_val", n_shap=100))
model.fit(project)

In [ ]:
analyser = model.components["ScoringAnalysis"]
shap_analyser = model.components["ShapAnalysis"]

In [ ]:

f, ax = plt.subplots(figsize=(9, 5), ncols=2)

plt.sca(ax[0])
analyser.plot_roc_curve(plot_method="beam")
analyser.plot_roc_curve(plot_method="mean")
plt.sca(ax[1]); analyser.plot_roc_curve(plot_method="all", cmap="rainbow")
plt.show()
print()

**[FOCUS] Explanantion tools**

1 . variable importance

In [ ]:
f, ax = plt.subplots(figsize=(8, 6), ncols=3, 
                     # sharey=True
                    )
plt.sca(ax[0])
analyser.plot_variable_importance(mode="minmax", color="blue")
plt.sca(ax[1])
analyser.plot_variable_importance(mode="std", color="lightgreen")
plt.sca(ax[2])
analyser.plot_variable_importance(mode="boxplot", cmap="rainbow_r")
plt.show()


print()

2. shap explanation 

In [ ]:
plt.figure(figsize=(5, 5)); shap_analyser.plot_shap_decision_plot()
plt.figure(figsize=(5, 5)); shap_analyser.plot_shap_summary_plot()

**Compute classification threshold**
Three methods are available to set the classification threshold

1. Corresponding to a provided fractile of the population targeted to one
2. Corresponding to a provided fractile of the positive true rate (recall) targeted to one
3. Corresponding to the optimized value of a provided metric

In [ ]:
print(analyser.compute_threshold(method="total_population", value=0.5))
print(analyser.compute_threshold(method="fpr", value=0.5))
print(analyser.compute_threshold(method="optimize_metric",  metric=metrics.accuracy_score))

**Confusion matrix** corresponding to target 50% of the total population

In [ ]:
analyser.compute_threshold(method="total_population", value=0.5)
analyser.confusion_matrix(in_percentage=True)

**Confusion matrix** corresponding to an accpetance of 50% of false positive rate 

In [ ]:
analyser.compute_threshold(method="fpr",value=0.5)
analyser.confusion_matrix(in_percentage=True)

In [ ]:
plt.figure(figsize=(6, 6))

analyser.plot_roc_curve(plot_method="all", cmap="rocket")
analyser.plot_threshold(label="Threshold (FPR = 0.5)")
plt.legend(loc=4)